<a href="https://colab.research.google.com/github/beastboy-93/wildecho/blob/main/Copy_of_imageipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: install all the necessary libraries for image classification

from google.colab import drive

drive.mount('/content/drive')

!pip install tensorflow
!pip install keras
!pip install opencv-python
!pip install matplotlib

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Define dataset paths
dataset_path = "/content/drive/MyDrive/footprints"
test_image_path = "/content/drive/MyDrive/test_fp/lion_1.jpg"
img_size = 128
num_classes = 10
model_path = "/content/drive/MyDrive/best_footprint.h5"

# Load and preprocess images
def load_images(dataset_path):
    X, y = [], []
    class_labels = sorted(os.listdir(dataset_path))  # Ensure consistent label

    for label_index, class_name in enumerate(class_labels):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            img = cv2.resize(img, (img_size, img_size))  # Resize to fixed dimensions
            img = img / 255.0  # Normalize pixel values

            X.append(img)
            y.append(label_index)

    X = np.array(X).reshape(-1, img_size, img_size, 1)  # Add channel dimension
    y = to_categorical(np.array(y), num_classes)  # One-hot encode labels
    return X, y, class_labels

# Load data
X, y, class_labels = load_images(dataset_path)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dataset loaded: {X_train.shape[0]} training samples, {X_test.shape[0]} testing samples")
print(class_labels)


# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

# Build CNN model
model = Sequential([
    Input(shape=(img_size, img_size, 1)),
    Conv2D(32, (3,3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2,2)),
    Conv2D(128, (3,3), activation='relu', padding='same'), BatchNormalization(), MaxPooling2D((2,2)),
    Flatten(),
    Dense(512, activation='relu'), BatchNormalization(), Dropout(0.5),
    Dense(256, activation='relu'), Dropout(0.3), Dense(num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
epochs = 50
batch_size = 16
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          validation_data=(X_test, y_test),
          epochs=epochs,
          callbacks=[ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, min_lr=1e-5, verbose=1),
                     EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, verbose=1),
                     ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, verbose=1)],
          verbose=1)

print(f"Model training complete and best model saved as '{model_path}'")

# Load best model
trained_model = load_model(model_path)


def predict_footprint(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Could not read image.")
        return None, None
    img = cv2.resize(img, (img_size, img_size))
    img_normalized = img / 255.0  # Normalize pixel values
    img_reshaped = np.array(img_normalized).reshape(1, img_size, img_size, 1)

    prediction = trained_model.predict(img_reshaped)
    predicted_class = np.argmax(prediction)
    confidence = np.max(prediction)

    return class_labels[predicted_class], confidence

# Function to predict footprint given an image path
def predict_from_image(image_file):
    predicted_label, confidence = predict_footprint(image_file)
    if predicted_label is None:
        return

    print(f"Predicted class: {predicted_label} with confidence: {confidence:.2f}")

# Example usage
predict_from_image(test_image_path)

Mounted at /content/drive
Dataset loaded: 400 training samples, 100 testing samples
['Bear', 'Bobcat', 'Deer', 'Fox', 'Horse', 'Lion', 'Mouse', 'Racoon', 'Squirrel', 'Wolf']
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.1058 - loss: 3.6216
Epoch 1: val_accuracy improved from -inf to 0.09000, saving model to /content/drive/MyDrive/best_footprint.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 57s 2s/step - accuracy: 0.1068 - loss: 3.6054 - val_accuracy: 0.0900 - val_loss: 2.2654 - learning_rate: 1.0000e-04
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1779 - loss: 2.5776
Epoch 2: val_accuracy did not improve from 0.09000
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.1775 - loss: 2.5844 - val_accuracy: 0.0900 - val_loss: 2.4190 - learning_rate: 1.0000e-04
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1747 - loss: 2.6980
Epoch 3: val_accuracy did not improve from 0.09000
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.1749 - loss: 2.6954 - val_accuracy: 0.0900 - val_loss: 2.6309 - learning_rate: 1.0000e-04
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2602 - loss: 2.3752
Epoch 4: val_accuracy did not improve from 0.09000
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.2584 - loss: 2.3828 - val_accuracy: 0.0900 - val_loss: 3.0909 - learning_rate: 1.0000e-04
Epoch 5/50
25/25 ━━━━━━━━━

25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.2215 - loss: 2.3258 - val_accuracy: 0.1100 - val_loss: 3.5773 - learning_rate: 5.0000e-05
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3021 - loss: 2.2261
Epoch 8: val_accuracy improved from 0.11000 to 0.12000, saving model to /content/drive/MyDrive/best_footprint.h5


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.3475 - loss: 2.1096 - val_accuracy: 0.3100 - val_loss: 1.9602 - learning_rate: 5.0000e-05
Epoch 25/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3263 - loss: 2.1193
Epoch 25: val_accuracy improved from 0.31000 to 0.33000, saving model to /content/drive/MyDrive/best_footprint.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.3260 - loss: 2.1164 - val_accuracy: 0.3300 - val_loss: 1.8870 - learning_rate: 5.0000e-05
Epoch 26/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2979 - loss: 2.1146
Epoch 26: val_accuracy improved from 0.33000 to 0.38000, saving model to /content/drive/MyDrive/best_footprint.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.2977 - loss: 2.1161 - val_accuracy: 0.3800 - val_loss: 1.8366 - learning_rate: 5.0000e-05
Epoch 27/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3324 - loss: 2.1031
Epoch 27: val_accuracy improved from 0.38000 to 0.46000, saving model to /content/drive/MyDrive/best_footprint.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.3324 - loss: 2.1006 - val_accuracy: 0.4600 - val_loss: 1.7105 - learning_rate: 5.0000e-05
Epoch 28/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2755 - loss: 2.2151
Epoch 28: val_accuracy did not improve from 0.46000
25/25 ━━━━━━━━━━━━━━━━━━━━ 31s 1s/step - accuracy: 0.2768 - loss: 2.2100 - val_accuracy: 0.4400 - val_loss: 1.7400 - learning_rate: 5.0000e-05
Epoch 29/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3443 - loss: 1.9584
Epoch 29: val_accuracy did not improve from 0.46000
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.3451 - loss: 1.9572 - val_accuracy: 0.4000 - val_loss: 1.8479 - learning_rate: 5.0000e-05
Epoch 30/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3386 - loss: 2.1298
Epoch 30: val_accuracy did not improve from 0.46000
25/25 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.3385 - loss: 2.1279 - val_accuracy: 0.3700 - val_loss: 1.7734 - learning_rate: 5.0000e-05
Epoch 31/50
25/25 ━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
Predicted class: Lion with confidence: 0.40


In [ ]:
print(class_labels)

['Bear', 'Bobcat', 'Deer', 'Fox', 'Horse', 'Lion', 'Mouse', 'Racoon', 'Squirrel', 'Wolf']
